In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from catboost import CatBoostClassifier
import optuna
import warnings
warnings.filterwarnings('ignore')

# Check scikit-learn version
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")

# Load data
train = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')
test = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')
test_ids = test["ID"]

# Clean data
train.drop(columns=["Unnamed: 0", "ID"], inplace=True, errors='ignore')
test.drop(columns=["Unnamed: 0", "ID"], inplace=True, errors='ignore')

X = train.drop(columns=["class"])
y = train["class"]

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
test_imputed = imputer.transform(test)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
test_scaled = scaler.transform(test_imputed)

# Feature selection
selector = SelectKBest(score_func=f_classif, k='all')
X_selected = selector.fit_transform(X_scaled, y_encoded)
test_selected = selector.transform(test_scaled)

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(X_selected, y_encoded, test_size=0.2, random_state=42)

# Define objective function for Optuna
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 300, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'random_seed': 42,
        'verbose': 0
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)
    return model.score(X_val, y_val)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Best parameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

# Train final model
model = CatBoostClassifier(**best_params, verbose=0)
model.fit(X_selected, y_encoded)

# Predict on test set
final_preds = model.predict(test_selected)
decoded_preds = label_encoder.inverse_transform(final_preds)

# Create and save submission
submission = pd.DataFrame({"ID": test_ids, "Predicted": decoded_preds})
submission.to_csv("submission_fast_catboost.csv", index=False)

# Validation score
val_score = model.score(X_val, y_val)
print(f"Validation accuracy: {val_score:.5f}")
print("✅ Submission file created: submission_fast_catboost.csv")
submission.head()

scikit-learn version: 1.2.2


[I 2025-06-14 18:34:43,495] A new study created in memory with name: no-name-dd7e3137-cb85-48ed-9da5-ced5f78576fe
[I 2025-06-14 18:34:49,989] Trial 0 finished with value: 0.955625 and parameters: {'iterations': 769, 'learning_rate': 0.11667169212810748, 'depth': 6, 'l2_leaf_reg': 9.698005984798408, 'border_count': 32}. Best is trial 0 with value: 0.955625.
[I 2025-06-14 18:35:11,913] Trial 1 finished with value: 0.953125 and parameters: {'iterations': 820, 'learning_rate': 0.028068979880741512, 'depth': 8, 'l2_leaf_reg': 3.188967225267848, 'border_count': 70}. Best is trial 0 with value: 0.955625.
[I 2025-06-14 18:35:16,368] Trial 2 finished with value: 0.959375 and parameters: {'iterations': 775, 'learning_rate': 0.08251075919718291, 'depth': 4, 'l2_leaf_reg': 1.808430005247462, 'border_count': 54}. Best is trial 2 with value: 0.959375.
[I 2025-06-14 18:35:37,673] Trial 3 finished with value: 0.955625 and parameters: {'iterations': 810, 'learning_rate': 0.07727857615552246, 'depth': 8

Best parameters: {'iterations': 684, 'learning_rate': 0.11873874584234431, 'depth': 7, 'l2_leaf_reg': 5.349883982355528, 'border_count': 72}
Validation accuracy: 0.99938
✅ Submission file created: submission_fast_catboost.csv


,ID,Predicted
0,1,forest
1,2,orchard
2,3,orchard
3,4,farm
4,5,orchard
